# Teaching markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [23]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [24]:
teaching = pd.read_excel('teaching.xlsx', 'teaching')
courses = pd.read_excel('teaching.xlsx', 'courses')

In [25]:
teaching.head(20)

,term,number,title,url_slug,venue,location
0,Fall 2007,MATH 112,College Algebra,class-1,Minnesota State University,"Mankato, MN"
1,Spring 2008,MATH 112,College Algebra,class-1,Minnesota State University,"Mankato, MN"
2,Fall 2008,MATH 112,College Algebra,class-1,Minnesota State University,"Mankato, MN"
3,Spring 2009,MATH 112,College Algebra,class-1,Minnesota State University,"Mankato, MN"
4,Fall 2009,MATH 140,College Algebra,class-1,Iowa State University,"Ames, IA"
5,Summer 2011,MATH 10,High School Algebra,class-2,Iowa State University,"Ames, IA"
6,Spring 2012,MATH 165,Calculus I,class-3,Iowa State University,"Ames, IA"
7,Summer 2012,MATH 166,Calculus II,class-4,Iowa State University,"Ames, IA"
8,Fall 2012,MATH 50,Calculus I,class-3,Drake Univeresity,"Des Moines, IA"
9,Spring 2013,MATH 166,Calculus II,class-4,Iowa State University,"Ames, IA"


In [26]:
teaching = teaching.merge(courses)
teaching

,term,number,title,url_slug,venue,location,description
0,Fall 2007,MATH 112,College Algebra,class-1,Minnesota State University,"Mankato, MN","Concepts of algebra (real numbers, exponents, ..."
1,Spring 2008,MATH 112,College Algebra,class-1,Minnesota State University,"Mankato, MN","Concepts of algebra (real numbers, exponents, ..."
2,Fall 2008,MATH 112,College Algebra,class-1,Minnesota State University,"Mankato, MN","Concepts of algebra (real numbers, exponents, ..."
3,Spring 2009,MATH 112,College Algebra,class-1,Minnesota State University,"Mankato, MN","Concepts of algebra (real numbers, exponents, ..."
4,Fall 2009,MATH 140,College Algebra,class-1,Iowa State University,"Ames, IA","Coordinate geometry, quadratic and polynomial ..."
...,...,...,...,...,...,...,...
70,Fall 2021,CDS 1010,Introduction to Programming,class-22,Hamline University,"Saint Paul, MN",Goals: To help students develop greater precis...
71,Spring 2020,CDS 3200,Elements of Statistical Learning,class-24,Hamline University,"Saint Paul, MN",Goals: This is a continuation course for MATH ...
72,Spring 2021,CDS 3200,Elements of Statistical Learning,class-24,Hamline University,"Saint Paul, MN",Goals: This is a continuation course for MATH ...
73,Spring 2020,MATH 3440,Discrete Mathematics,class-25,Hamline University,"Saint Paul, MN",Goals: To introduce the concept of the discret...


In [27]:
def term_order(term):
    if term == 'Fall':
        return '04'
    elif term in ['J-term', 'Winter']:
        return '01'
    elif term == 'Spring':
        return '02'
    elif term == 'Summer':
        return '03'

teaching['year'] = teaching.apply(lambda row: row.term.split(' ')[1], axis=1)
teaching['term'] = teaching.apply(lambda row: row.term.split(' ')[0], axis=1)
teaching['term_order'] = teaching.apply(lambda x: term_order(x.term), axis=1)

teaching = teaching[['term', 'year', 'term_order', 'number', 'title', 'url_slug', 'venue', 'location', 'description']]

In [28]:
teaching

,term,year,term_order,number,title,url_slug,venue,location,description
0,Fall,2007,04,MATH 112,College Algebra,class-1,Minnesota State University,"Mankato, MN","Concepts of algebra (real numbers, exponents, ..."
1,Spring,2008,02,MATH 112,College Algebra,class-1,Minnesota State University,"Mankato, MN","Concepts of algebra (real numbers, exponents, ..."
2,Fall,2008,04,MATH 112,College Algebra,class-1,Minnesota State University,"Mankato, MN","Concepts of algebra (real numbers, exponents, ..."
3,Spring,2009,02,MATH 112,College Algebra,class-1,Minnesota State University,"Mankato, MN","Concepts of algebra (real numbers, exponents, ..."
4,Fall,2009,04,MATH 140,College Algebra,class-1,Iowa State University,"Ames, IA","Coordinate geometry, quadratic and polynomial ..."
...,...,...,...,...,...,...,...,...,...
70,Fall,2021,04,CDS 1010,Introduction to Programming,class-22,Hamline University,"Saint Paul, MN",Goals: To help students develop greater precis...
71,Spring,2020,02,CDS 3200,Elements of Statistical Learning,class-24,Hamline University,"Saint Paul, MN",Goals: This is a continuation course for MATH ...
72,Spring,2021,02,CDS 3200,Elements of Statistical Learning,class-24,Hamline University,"Saint Paul, MN",Goals: This is a continuation course for MATH ...
73,Spring,2020,02,MATH 3440,Discrete Mathematics,class-25,Hamline University,"Saint Paul, MN",Goals: To introduce the concept of the discret...


In [29]:
teaching.sort_values(by=['year', 'term_order', 'number'], axis=0).head(20)

,term,year,term_order,number,title,url_slug,venue,location,description
0,Fall,2007,04,MATH 112,College Algebra,class-1,Minnesota State University,"Mankato, MN","Concepts of algebra (real numbers, exponents, ..."
1,Spring,2008,02,MATH 112,College Algebra,class-1,Minnesota State University,"Mankato, MN","Concepts of algebra (real numbers, exponents, ..."
2,Fall,2008,04,MATH 112,College Algebra,class-1,Minnesota State University,"Mankato, MN","Concepts of algebra (real numbers, exponents, ..."
3,Spring,2009,02,MATH 112,College Algebra,class-1,Minnesota State University,"Mankato, MN","Concepts of algebra (real numbers, exponents, ..."
4,Fall,2009,04,MATH 140,College Algebra,class-1,Iowa State University,"Ames, IA","Coordinate geometry, quadratic and polynomial ..."
5,Summer,2011,03,MATH 10,High School Algebra,class-2,Iowa State University,"Ames, IA","Topics include signed numbers, polynomials, ra..."
6,Spring,2012,02,MATH 165,Calculus I,class-3,Iowa State University,"Ames, IA","Differential calculus, applications of the der..."
7,Summer,2012,03,MATH 166,Calculus II,class-4,Iowa State University,"Ames, IA","Integral calculus, applications of the integra..."
9,Fall,2012,04,MATH 50,Calculus I,class-3,Drake Univeresity,"Des Moines, IA","Very brief review of algebra, logarithms, and ..."
8,Spring,2013,02,MATH 166,Calculus II,class-4,Iowa State University,"Ames, IA","Integral calculus, applications of the integra..."


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [30]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

In [31]:
for row, item in teaching.iterrows():
    print(item)

term                                                        Fall
year                                                        2007
term_order                                                    04
number                                                  MATH 112
title                                            College Algebra
url_slug                                                 class-1
venue                                 Minnesota State University
location                                             Mankato, MN
description    Concepts of algebra (real numbers, exponents, ...
Name: 0, dtype: object
term                                                      Spring
year                                                        2008
term_order                                                    02
number                                                  MATH 112
title                                            College Algebra
url_slug                                                 class-1
ve

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [32]:
loc_dict = {}

for row, item in teaching.iterrows():
    
    # if '/' in str(item.number):
    #     tmp_number = item.number.replace('/', '')
    #     md_filename = str(tmp_number).replace(' ', '_') + '-' + item.url_slug + '.md'
    #     html_filename = str(tmp_number).replace(' ', '_') + "-" + item.url_slug 
    # else:
    #     md_filename = str(item.number).replace(' ', '_') + "-" + item.url_slug + ".md"
    #     html_filename = str(item.number).replace(' ', '_') + "-" + item.url_slug 

    md_filename = str(item.term) + '-' + str(item.year) + "-" + item.url_slug + ".md"
    html_filename = str(item.term) + '-'  + str(item.year) + "-" + item.url_slug 

    year = item.year
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: teaching" + "\n"
    
    md += 'type: "teaching"\n'
    
    md += "permalink: /teaching/" + html_filename + "\n"
        
    if len(str(item.term)) > 3:
        md += "date: " + str(item.term) + str(item.year) + "\n"
        
    if len(str(item.number)) > 3:
        md += "number: " + str(item.number) + "\n"
        
    if len(str(item.url_slug)) > 3:
        md += "url_slug: " + str(item.url_slug) + "\n"
        
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
#     if len(str(item.talk_url)) > 3:
#         md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_teaching/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [33]:
!ls ../_teaching

CDS_1010-class-22.md  MATH_112-class-1.md   MATH_310-class-15.md
CDS_3200-class-24.md  MATH_116-class-12.md  MATH_3320-class-18.md
CDS_5950-class-26.md  MATH_1170-class-3.md  MATH_3440-class-25.md
CSCI_1250-class-22.md MATH_1200-class-20.md MATH_351-class-10.md
CSCI_1980-class-19.md MATH_140-class-1.md   MATH_450-class-16.md
FSEM_1010-class-23.md MATH_165-class-3.md   MATH_50-class-3.md
MATH_094-class-9.md   MATH_166-class-4.md   MATH_5950-class-21.md
MATH_095-class-11.md  MATH_261-class-17.md  MATH_70-class-4.md
MATH_096-class-6.md   MATH_2667-class-5.md
MATH_10-class-2.md    MATH_300-class-13.md


In [34]:
!cat ../_teaching/MATH_70-class-4.md

---
title: "Calculus II"
collection: teaching
type: "teaching"
permalink: /teaching/MATH_70-class-4
date: Spring2014
number: MATH 70
url_slug: class-4
venue: "Drake Univeresity"
location: "Des Moines, IA"
---

Advanced applications of differentiation; advanced techniques and applications of integration; simple examples of differential equations.


In [35]:
!cat ../_teaching/CDS_1010-class-22.md

---
title: "Introduction to Programming"
collection: teaching
type: "teaching"
permalink: /teaching/CDS_1010-class-22
date: Fall2021
number: CDS 1010
url_slug: class-22
venue: "Hamline University"
location: "Saint Paul, MN"
---

Goals: To help students develop greater precision in their algorithmic thinking by writing moderate-sized programs for a variety of applications, including but not limited to biology, chemistry,  economics, literary studies, and mathematics.

Content: Students will learn the fundamentals of computer programming (loop structures, if-else statements, Boolean expressions, and arrays) to solve  problems from different disciplines. A short introduction to object-oriented programming is also given. This course is taught using Python.


In [36]:
!cat ../_teaching/FSEM_1010-class-23.md

---
title: "First Year Seminar: Uses and Misuses of Algorithms"
collection: teaching
type: "teaching"
permalink: /teaching/FSEM_1010-class-23
date: Fall2021
number: FSEM 1010
url_slug: class-23
venue: "Hamline University"
location: "Saint Paul, MN"
---

Data scientists have used algorithms for many great things: Netflix’s recommendation system; building teams that go on to win the World Series or the Stanley Cup; proving the existence of the Higgs boson; and early detection of cancer. Other data scientists have used algorithms with malicious intent: targeting of vulnerable people by payday loan companies and for-profit higher education companies that provide little---if any---benefit to their students; using of social network bots to spread misinformation and sow discontent within a country. Sometimes algorithms have unintended negative effects: the firing of skilled teachers in Washington, D.C.; racial discrimination in the lengths of prison sentences and the granting (or not granting